In [1]:
from hydradx.model.amm.omnipool_amm import OmnipoolState
from hydradx.model.amm.global_state import GlobalState, liquidate_against_omnipool
from hydradx.model.amm.agents import Agent
from hydradx.model.amm.liquidations import CDP
from hydradx.model import run



In [2]:

# same seed, same parameters = same simulation result
# random.seed(42)

prices = {'DOT': 6, 'HDX': 0.02, 'USDT': 1, 'WETH': 2500, 'iBTC': 45000}

assets = {
    'DOT': {'usd price': prices['DOT'], 'weight': 0.40},
    'HDX': {'usd price': prices['HDX'], 'weight': 0.10},
    'USDT': {'usd price': prices['USDT'], 'weight': 0.30},
    'WETH': {'usd price': prices['WETH'], 'weight': 0.10},
    'iBTC': {'usd price': prices['iBTC'], 'weight': 0.10}
}

lrna_price_usd = 35
initial_omnipool_tvl = 20000000
liquidity = {}
lrna = {}

for tkn, info in assets.items():
    liquidity[tkn] = initial_omnipool_tvl * info['weight'] / info['usd price']
    lrna[tkn] = initial_omnipool_tvl * info['weight'] / lrna_price_usd
    
cdps = [
    CDP('USDT', 'DOT', 100, 20, True),
    CDP('USDT', 'HDX', 100, 10000, False),
    CDP('USDT', 'WETH', 25000000, 10200, True),
]

initial_state = GlobalState(
    pools={
        'omnipool': OmnipoolState(
            tokens={
                tkn: {'liquidity': liquidity[tkn], 'LRNA': lrna[tkn]} for tkn in assets
            },
            preferred_stablecoin='USDT',
        )
    },
    agents = {'Treasury': Agent(holdings={tkn: 0 for tkn in assets})},
    cdps=cdps,
    archive_all=True,
    evolve_function=liquidate_against_omnipool('omnipool', 'Treasury'),
)

# print(f'Running with {3} time steps')
events = run.run(initial_state, time_steps=2)

Starting simulation...
Execution time: 0.012 seconds.


In [3]:
from pprint import pprint

pprint(cdps)
for event in events:
    pprint(event.cdps)
    pprint(event.agents['Treasury'].holdings)

[CDP(USDT, DOT, 100, 20, True),
 CDP(USDT, HDX, 100, 10000, False),
 CDP(USDT, WETH, 25000000, 10200, True)]
[CDP(USDT, DOT, 0.0, 3.16617077011394, True),
 CDP(USDT, HDX, 100, 10000, False),
 CDP(USDT, WETH, 24985480.308532715, 10194.075928214546, True)]
{'DOT': 0, 'HDX': 0, 'USD': 0, 'USDT': 146.19691467285156, 'WETH': 0, 'iBTC': 0}
[CDP(USDT, DOT, 0.0, 3.16617077011394, True),
 CDP(USDT, HDX, 100, 10000, False),
 CDP(USDT, WETH, 24985480.308532715, 10194.075928214546, True)]
{'DOT': 0, 'HDX': 0, 'USD': 0, 'USDT': 146.19691467285156, 'WETH': 0, 'iBTC': 0}
